In [3]:
pwd

'/Users/edwardterrell/Desktop/Training/kindle/strava'

In [1]:
import requests
import json
import time
import os
from pprint import pprint
import pandas as pd
pd.set_option('display.precision', 1)
from clean_convert import cleanup, convert_speed

In [2]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))
import sys
sys.executable

'/Users/edwardterrell/opt/anaconda3/envs/mlml/bin/python'

In [4]:
from token_manager import get_valid_access_token

access_token = get_valid_access_token()
print("Using access token:", access_token)

# Make an authenticated API call
response = requests.get(
    'https://www.strava.com/api/v3/athlete',
    headers={'Authorization': f'Bearer {access_token}'}
)
print(response.json())

Using access token: 8e5e6a3dd85ce84a4003a72c0a7b1b163bb2d747
{'id': 723727, 'username': 'eterrell', 'resource_state': 2, 'firstname': 'Ed', 'lastname': 'Terrell', 'bio': '', 'city': 'Boulder', 'state': 'CO', 'country': 'United States', 'sex': 'M', 'premium': True, 'summit': True, 'created_at': '2012-07-05T00:04:09Z', 'updated_at': '2025-07-09T12:38:59Z', 'badge_type_id': 1, 'weight': 61.235, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/723727/20826329/1/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/723727/20826329/1/large.jpg', 'friend': None, 'follower': None}


In [ ]:
# USED ONLY IF token_manager.py isn't generating correctly
client_id = 167221
client_secret = 'f4df8e0dec7d9d8e725707eaf05037fcc845989d'
auth_code = 'd973f0f86d2e56c3e7ac369809ca6b14fc814a98' # stored from last time

response = requests.post(
    'https://www.strava.com/oauth/token',
    data={
        'client_id': client_id,
        'client_secret': client_secret,
        'code': auth_code,
        'grant_type': 'authorization_code',
    }
)
# Check and save response
if response.status_code == 200:
    tokens = response.json()
    print("✅ Tokens received:")
    print(json.dumps(tokens, indent=2))

    # Save to tokens.json for use in token_manager.py
    with open("tokens.json", "w") as f:
        json.dump(tokens, f)
else:
    print("❌ Failed to get tokens:", response.status_code)
    print(response.text)

In [ ]:
import datetime
expires_at = 1751931129
# Convert to datetime object (local time)
dt_local = datetime.datetime.fromtimestamp(expires_at)
print("Local time: ", dt_local)

In [5]:
# Get recent activities
response = requests.get(
    'https://www.strava.com/api/v3/athlete/activities',
    headers={'Authorization': f'Bearer {access_token}'}
)
activities = response.json()
response.status_code

200

In [6]:
# Show the first 3 activities with ID and name
for activity in activities[:3]:
    print(f"ID: {activity['id']}, Name: {activity['name']}, Date: {activity['start_date']}")

ID: 15061307314, Name: Nederland exploration and exploitation, Date: 2025-07-09T14:17:28Z
ID: 15048399185, Name: Jamestown and eats at Lucky's Bakery, Date: 2025-07-08T14:44:02Z
ID: 15037283785, Name: Old Stage, Date: 2025-07-07T12:38:38Z


### Strava_df, Build

In [7]:
# Build out strava_df with only the necessary columns
strava_df = pd.DataFrame(activities)
cols_needed = ['id','name', 'distance', 'moving_time','total_elevation_gain', 'sport_type',
          'average_speed', 'average_heartrate','max_heartrate', 'suffer_score','start_date']
strava_df = strava_df.loc[:,cols_needed]

strava_df = strava_df.dropna(subset=['average_heartrate'])

strava_df.head(2)

,id,name,distance,moving_time,total_elevation_gain,sport_type,average_speed,average_heartrate,max_heartrate,suffer_score,start_date
0,15061307314,Nederland exploration and exploitation,54140.4,11638,993.7,Ride,4.7,109.1,131.0,106.0,2025-07-09T14:17:28Z
1,15048399185,Jamestown and eats at Lucky's Bakery,60380.4,9467,757.0,Ride,6.4,110.9,140.0,101.0,2025-07-08T14:44:02Z


In [ ]:
def cleanup (strava_df):
    """
    Clean and transform Strava activity DataFrame:
    - Convert start_date to datetime and extract date
    - Convert distance from meters to miles
    - Convert moving_time to minutes and add formatted time (hh:mm)
    - Convert elevation gain from meters to feet
    """
    # Change to datetime and create new date column
    strava_df.start_date = pd.to_datetime(strava_df.start_date) 
    strava_df['date'] = strava_df.start_date.dt.date
    # Change distance to miles
    strava_df['distance'] = strava_df['distance'] / 1609.34
    # Change moving time to minutes in new col: time
    strava_df['time'] = strava_df.moving_time/60
    # Convert to timedelta
    strava_df.moving_time = pd.to_timedelta(strava_df['moving_time'], unit='s')
    # Change to hours/minutes
    strava_df['moving_time'] = strava_df['moving_time'].apply(lambda td: f"{int(td.total_seconds() // 3600)}:{int((td.total_seconds() % 3600) // 60):02}")
    # Change to feet
    strava_df.total_elevation_gain = strava_df.total_elevation_gain*3.28084
    
    return strava_df

def convert_speed(row):
    if row['sport_type'] == 'Ride':
        return row['average_speed'] * 2.23694  # m/s to mph
    elif row['sport_type'] == 'Run':
        if row['average_speed'] > 0:
            pace_sec_per_mile = 1609.34 / row['average_speed']
            minutes = int(pace_sec_per_mile // 60)
            seconds = int(pace_sec_per_mile % 60)
            return f"{minutes}:{seconds:02d}"
        else:
            return "N/A"
    else:
        return None  # for other sports

In [8]:
# process data
cleanup (strava_df)

# add column: converted_speed -mph (ride) and -min/mile (run)
strava_df['converted_speed'] = strava_df.apply(convert_speed, axis=1)

# Reorder columns
new_order = ['id', 'name','date','sport_type',  'distance','moving_time', 'converted_speed', 
       'total_elevation_gain', 'average_heartrate', 'max_heartrate',
       'suffer_score', 'time','average_speed','start_date' ]

strava_df = strava_df[new_order]

# modify column names
strava_df.columns = ['id', 'name', 'date', 'sport', 'distance', 'time',
       'speed', 'elev_gain',
       'avg_HR', 'max_HR', 'suffer_score', 'time-minutes','average_speed',
       'start_date']

In [9]:
strava_df.head(2)

,id,name,date,sport,distance,time,speed,elev_gain,avg_HR,max_HR,suffer_score,time-minutes,average_speed,start_date
0,15061307314,Nederland exploration and exploitation,2025-07-09,Ride,33.6,3:13,10.4,3260.2,109.1,131.0,106.0,194.0,4.7,2025-07-09 14:17:28+00:00
1,15048399185,Jamestown and eats at Lucky's Bakery,2025-07-08,Ride,37.5,2:37,14.3,2483.6,110.9,140.0,101.0,157.8,6.4,2025-07-08 14:44:02+00:00


### Using the dataframe approach

In [10]:
# Create a zone dataframe
cols = ['id', 'name', 'date', 'suffer_score']
data = strava_df.iloc[:,[0,1,2,-4]]
strava_zone_df = pd.DataFrame(data, columns=cols)
strava_zone_df.tail()

,id,name,date,suffer_score
25,14694429401,3 mile Morning Run,2025-06-04,52.0
26,14684468262,SuperFlag,2025-06-03,160.0
27,14672337121,Boulder Res run - recovery day,2025-06-02,84.0
28,14666228613,SuperJames and Ward,2025-06-01,281.0
29,14621663272,The Long Way Home,2025-05-28,230.0


In [11]:
# activity ID to inspect
# activity_id = 15048399185

def get_zones_for_id(activity_id):
    print(activity_id)
    # endpoint URL
    url = f'https://www.strava.com/api/v3/activities/{activity_id}/zones'
    # Set up Authorization header and make request
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        zones = response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
    # create time_in_zones series
    zone_series  = pd.DataFrame(zones[0]).iloc[:,1]
    time_in_zones = zone_series.apply(lambda z: z['time']/60)
    time_in_zones.index = ['Zone1', 'Zone2', 'Zone3', 'Zone4', 'Zone5']
    time.sleep(1.1)
    return (time_in_zones)

In [12]:
zone_df = strava_zone_df['id'].apply(get_zones_for_id)

15061307314
15048399185
15037283785
15027982989
15017668877
15006491752
14975645989
14964904782
14954454448
14944687171
14934728422
14925456666
14915319034
14873916201
14860197741
14854414147
14840593381
14767120578
14758807206
14748055178
14737389332
14727760782
14715424784
14705554493
14694429401
14684468262
14672337121
14666228613
14621663272


In [13]:
zone_df['moderate'] = zone_df[['Zone1', 'Zone2']].sum(axis=1)
zone_df['intense'] = zone_df[['Zone3', 'Zone4', 'Zone5']].sum(axis=1)
# zone_df.head()
# concat both dfs
strava_zone_df = pd.concat([strava_zone_df, zone_df], axis=1)
strava_zone_df